In [2]:
import pennylane as qml
from pennylane import numpy as np

In [3]:
WIRES = 2  # Número de hilos o qubits
LAYERS = 5  # Número de capas en el circuito
NUM_PARAMETERS = LAYERS * WIRES * 3  # Número total de parámetros en el circuito

In [4]:
# Definiendo el circuito cuántico variacional dado por Pennylane
def variational_circuit(params, hamiltonian):
    # Reestructurando los parámetros para que coincidan con el número de capas y qubits
    parameters = params.reshape((LAYERS, WIRES, 3))
    # Aplicando capas de entrelazamiento fuerte (StronglyEntanglingLayers) al circuito
    qml.templates.StronglyEntanglingLayers(parameters, wires=range(WIRES))
    # Retornando el valor esperado del hamiltoniano
    return qml.expval(qml.Hermitian(hamiltonian, wires=[0, 1]))

In [5]:
# Función para optimizar el circuito cuántico dado un hamiltoniano
def optimize_circuit(hamiltonian):
    # Convirtiendo el hamiltoniano en un array y asegurando que no requiere gradiente
    hamiltonian = np.array(hamiltonian, requires_grad=False)
    # Reestructurando el hamiltoniano para que tenga la forma adecuada (4x4 en este caso)
    hamiltonian = np.array(hamiltonian, float).reshape((2 ** WIRES), (2 ** WIRES))
    # Definiendo el dispositivo cuántico
    dev = qml.device('default.qubit', wires=WIRES)
    # Creando el nodo cuántico del circuito
    circuit = qml.QNode(variational_circuit, dev)
    # Inicializando parámetros aleatoriamente para el circuito
    init_params = np.random.randn(NUM_PARAMETERS)
    # Función de costo que queremos minimizar (valor esperado del hamiltoniano)
    def cost(params):
        return circuit(params, hamiltonian)
    # Definiendo el optimizador (Descenso de Gradiente)
    optimizer = qml.GradientDescentOptimizer(stepsize=0.4)
    num_steps = 200  # Número de pasos de optimización
    params = init_params  # Iniciando con los parámetros aleatorios
    # Loop de optimización
    for _ in range(num_steps):
        params = optimizer.step(cost, params)
    # Retornando el valor del costo optimizado
    return cost(params)

In [6]:
min_value = float('inf')
# Definiendo un ejemplo de hamiltoniano
hamiltonian_example = np.array(
    [0.863327072347624,0.0167108057202516,0.07991447085492759,0.0854049026262154,
     0.0167108057202516,0.8237963773906136,-0.07695947154193797,0.03131548733285282,
     0.07991447085492759,-0.07695947154193795,0.8355417021014687,-0.11345916130631205,
     0.08540490262621539,0.03131548733285283,-0.11345916130631205,0.758156886827099])
# Optimiza el circuito con el hamiltoniano 
optimized_value = optimize_circuit(hamiltonian_example)
print("Optimized expectation value:", optimized_value)

Optimized expectation value: 0.6174534093371777


In [7]:
min_value = float('inf')
# Definiendo un ejemplo de hamiltoniano
hamiltonian_example = np.array(
    [0.32158897156285354,-0.20689268438270836,0.12366748295758379,-0.11737425017261123,
     -0.20689268438270836,0.7747346055276305,-0.05159966365446514,0.08215539696259792,
     0.12366748295758379,-0.05159966365446514,0.5769050487087416,0.3853362904758938,
     -0.11737425017261123,0.08215539696259792,0.3853362904758938,0.3986256655167206])
# Optimiza el circuito con el hamiltoniano 
optimized_value = optimize_circuit(hamiltonian_example)
print("Optimized expectation value:", optimized_value)

Optimized expectation value: 0.0024648812008860738
